In [3]:
import nltk
from numpy import array, ones, zeros, multiply
import numpy as np
import sys

UNK = "<unk>"  # token to map all out-of-vocabulary words (OOVs)
UNKid = 0      # index for UNK
epsilon=1e-100

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, smoothing_obs = 0.01):
            """
            Builds a Hidden Markov Model
            * state_list is the list of state symbols [q_0...q_(N-1)]
            * observation_list is the list of observation symbols [v_0...v_(M-1)]
            * transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            * observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            * initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print "HMM creating with: "
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            UNKid = self.M+1;                   # should not correspond to zeroth index
            print str(self.N)+" states"
            print str(self.M)+" observations"
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M+1, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
            
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices

    
        def data2indices(self, sent): 
            """From one (letter,correction) pair 
            - extract the letter and correction 
            - returns two list of indices, one for each
            -> (letterid, correctionid)
            """
            letterids = list()
            correctionids  = list()
            for couple in sent:
                letter = couple[0]
                correction = couple[1]
                if letter in self.X_index:
                    letterids.append(self.X_index[letter])
                else:
                    letterids.append(UNKid)
                correctionids.append(self.Y_index[correction])
            return letterdids, correctionids
            
        def observation_estimation(self, pair_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            # fill with counts
            for pair in pair_counts:
                wrd=pair[0]
                tag=pair[1]
                cpt=pair_counts[pair]
                k = self.M # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                i=self.Y_index[tag]
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)
            
        
        def transition_estimation(self, trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
            """
            # fill with counts
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index[pair[0]]
                self.transition_proba[i,j]=trans_counts[pair]
            # normalize
            self.transition_proba=self.transition_proba/self.transition_proba.sum(axis=0).reshape(1,self.N)
        
        def init_estimation(self, init_counts):
            """Build the init. distribution"""
            # fill with counts
            for tag in init_counts:
                i=self.Y_index[tag]
                self.initial_state_proba[i]=init_counts[tag]
            # normalize
            self.initial_state_proba=self.initial_state_proba/sum(self.initial_state_proba)
             
        
        def supervised_training(self, pair_counts, trans_counts,init_counts):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.init_estimation(init_counts)
            
        def forward(self, obs):  
            leng=len(obs)
            alpha = zeros((leng,self.N),float)            
            for y in range(self.N):
                alpha[0][y] = self.initial_state_proba[y] * self.observation_proba[self.X_index[obs[0]]][y]
            for t in range(1, leng):  
                k=self.X_index[obs[t]]
                for y in range(self.N):                    
                    alpha[t][y] = sum((alpha[t-1][y0] * self.transition_proba[y0][y] * self.observation_proba[k][y]) for y0 in range(self.N))
            
            return alpha
        
        def backward(self,obs):
            beta=zeros((len(obs),self.N),float)
            leng = len(obs)
            for y in range(self.N):
                beta[T-1][y] = 1 
            for t in reversed(range(T-1)):
                k=self.X_index[obs[t]]
                for y in range(self.N):
                    beta[t][y] = sum((beta[t+1][y1] * self.transition_proba[y][y1] * self.B[k][y1]) for y1 in range(self.N))

            return beta
        
        def forward_backward(self,obs):
            n = len(obs)
            m = self.N

            alpha = self.forward(obs)
            beta = self.backward(obs)
            gamma = np.zeros((n, m), float)
            for t in xrange(n):
                gamma[t, :] = [alpha[t, i] * beta[t, i] for i in xrange(m)]
                gamma[t, :] /= np.sum(gamma[t, :])

            return alpha, beta, gamma
 

# Compter les mots et les tags

In [4]:
def make_counts(corpus):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns: 
    * c_words: word counts
    * c_tags: tag counts
    * c_pairs: count of pairs (word,tag)
    * c_transitions: count of tag bigram 
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_inits = dict()
    for sent in corpus:
        # we use i because of the transition counts
        for i in range(len(sent)):
            couple=sent[i]
            wrd = couple[0]
            tag = couple[1]
            # word counts
            if wrd in c_words:
                c_words[wrd]=c_words[wrd]+1
            else:
                c_words[wrd]=1
            # tag counts
            if tag in c_tags:
                c_tags[tag]=c_tags[tag]+1
            else:
                c_tags[tag]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            if i > 0:
                trans = (sent[i-1][1],tag)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
            # i == 0 -> counts for initial states
            else:
                if tag in c_inits:
                    c_inits[tag]=c_inits[tag]+1
                else:
                    c_inits[tag]=1
                    
    return c_words,c_tags,c_pairs, c_transitions, c_inits


# les données


In [31]:
import cPickle as pickle
train=pickle.load( open( "D:/GitHub/HMM_typos_correction-/data/train10.pkl", "rb" ))
c_words,c_tags,c_pairs, c_transitions, c_inits = make_counts(train)

# Création du HMM et APPRENTISAGE

In [32]:
hmm = HMM(state_list=c_tags.keys(), observation_list=c_words.keys(),
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                 smoothing_obs = 0.001)
hmm.supervised_training(c_pairs,c_transitions,c_inits)
del train

HMM creating with: 
26 states
26 observations


In [9]:
def find_states(Xd,Pi,A,B,hmm):
	# 'allq' stands for current estimated hidden_states

	probs=[]
	states=[]	#new hidden-states given observations Xc
	for x in Xd:
		seq = []
        	for letter in x:
            		seq.append(hmm.X_index[letter[0]])
		state,prob = viterbi(seq,Pi,A,B)	#finding states and most likely path given these models(i.e fixed Pi,A,B)
		states.append(state)
		probs.append(prob)

	return probs,states



In [11]:
def viterbi(x,Pi,A,B):

	#Initialization
	T = len(x)
	N = len(Pi) # no. of states
	delta_0 = []
	for i in range(N):
		delta_0.append ( np.log(Pi[i]) + np.log(B[i,x[0]]) )
	phi_0 = np.ones(Pi.shape)*-1

	#Recursion
	Phi = []
	previous_delta = delta_0
	for i in range(1,T):

		delta_t = []
		for j in range(N):
			delta_t.append( max( previous_delta+np.log(A[:,j]) ) + np.log(B[j,x[i]]) )

		phi_t = []
		for j in range(N):
			phi_tj = np.argmax( previous_delta+np.log(A[:,j]) )
			phi_t.append( phi_tj )

		Phi.append(phi_t)
		previous_delta = delta_t

	#Termination
	S_star = max(previous_delta) #probablity of given observations

	#backward tracing of the states
	states_in_inverse_order = []
	final_state = np.argmax(previous_delta)
	states_in_inverse_order.append(final_state)
#	pdb.set_trace()
	for i in range(T-2,-1,-1):
#		pdb.set_trace()
		final_state = Phi[i][final_state]				
		states_in_inverse_order.append(final_state)
	return states_in_inverse_order,S_star

In [38]:
Pi = hmm.initial_state_proba
A = hmm.transition_proba
B = hmm.observation_proba
A = A.T  #code assumes that row-wise probabilities sum to 1
B = B.T
# Data is assumed to comes in chains of observation, so recovering each chain of observation
Xd=[]
allq=[] #denotes all underlying hidden states
test=pickle.load( open( "D:/GitHub/HMM_typos_correction-/data/test20.pkl", "rb" ))
for sent in test:
    data = np.asarray(sent)
    obs,states = np.hsplit(data,2)
    Xd.append(obs)
    allq.append(states)
del test
Xd = np.array(Xd)
allq = np.array(allq)   #These are the true lables

In [21]:
def compute_error(corrections,true_vals):
    """Compares the corrections and true_vals"""
    error=0
    total=0
    for f, b in zip(corrections, true_vals):
        if cmp(f,b)!=0:
            for i in range(len(f)):
                if f[i]!=b[i]:
                    error+=1
        total+=len(f)

    return float(error)/float(total)                    

In [39]:
probs,allq_est =  find_states(Xd,Pi,A,B,hmm)    # Running the Viterbi

D:\Programs\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: divide by zero encountered in log
D:\Programs\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: divide by zero encountered in log
D:\Programs\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: divide by zero encountered in log


In [40]:
# Converting the true letters into their corresponding-index
truevals=[]
for trueval in allq:
    val=[]
    for l in trueval:
        val.append(hmm.Y_index[l[0]])
    truevals.append(val)

In [41]:
# Inversing the states-retrived from Viterbi (as they are recovered in inverse order)
corrections=[]
for q in allq_est:
    q.reverse()
    corrections.append(q)

In [42]:
print compute_error(corrections,truevals)

0.13234677371
